In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from pykrx import stock
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
from sklearn.linear_model import LassoCV, RidgeCV
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import GradientBoostingClassifier
from tqdm.auto import tqdm

# 데이터 로드 및 전처리 (위의 예제와 동일한 방식으로)
df = pd.read_csv("하이닉스 power (3).csv", thousands=',', encoding='cp949')
df['next_day_return'] = (df['종가'].shift(-1) - df['종가']) / df['종가'] * 100
df['target'] = df['next_day_return'].apply(lambda x: 1 if x > 0.25 else 0)
df.dropna(inplace=True)

X = df.drop(['날짜', 'target', 'next_day_return'], axis=1)
y = df['target']
# Random Forest for feature importance
rf = RandomForestRegressor()
rf.fit(X, y)

features_rf_sorted = sorted(zip(X.columns, rf.feature_importances_), key=lambda x: x[1], reverse=True)
features_rf_df = pd.DataFrame(features_rf_sorted, columns=['Feature', 'RF Importance'])

# 상관관계가 가장 높은 상위 8개의 피처를 선정
top_7_features = [feature for feature, importance in features_rf_sorted[:7]]
# X와 y 정의 (df2와 이전 코드에서의 처리 과정을 바탕으로)dp
X_top7 = X[top_7_features]
y = df['target']  # y 값을 0과 1로 조정

# 데이터를 학습 세트와 테스트 세트로 분할
X_train, X_test, y_train, y_test = train_test_split(X_top7, y, test_size=0.25, random_state=66)

# 로지스틱 회귀 모델을 위한 하이퍼파라미터 그리드 정의
param_grid_lr = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],  # 규제 강도의 역수
    'penalty': ['l1', 'l2', 'elasticnet', 'none'],  # 사용할 규제
    'solver': ['saga']  # 'saga' 솔버는 모든 penalty 옵션 지원
}

# GridSearchCV 객체 초기화 및 학습
grid_search_lr = GridSearchCV(LogisticRegression(max_iter=10000, random_state=1), param_grid_lr, refit=True, verbose=2, cv=5, n_jobs=-1)
grid_search_lr.fit(X_train, y_train)

# 최적의 하이퍼파라미터와 모델 출력
print(f"Best parameters: {grid_search_lr.best_params_}")
best_lr_model = grid_search_lr.best_estimator_

# 최적의 모델로 테스트 데이터 예측
y_pred_lr = best_lr_model.predict(X_test)

# 예측 결과에 대한 평가 지표 출력
accuracy_lr = accuracy_score(y_test, y_pred_lr)
precision_lr = precision_score(y_test, y_pred_lr, average='macro')
recall_lr = recall_score(y_test, y_pred_lr, average='macro')
f1_lr = f1_score(y_test, y_pred_lr, average='macro')

print(f"Accuracy (LR): {accuracy_lr:.4f}")
print(f"Precision (LR): {precision_lr:.4f}")
print(f"Recall (LR): {recall_lr:.4f}")
print(f"F1 Score (LR): {f1_lr:.4f}")

Fitting 5 folds for each of 24 candidates, totalling 120 fits
Best parameters: {'C': 0.001, 'penalty': 'l1', 'solver': 'saga'}
Accuracy (LR): 0.5382
Precision (LR): 0.5296
Recall (LR): 0.5298
F1 Score (LR): 0.5296


c:\Users\yeotaekyeong\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
60 fits failed out of a total of 120.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
30 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\yeotaekyeong\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\yeotaekyeong\anaconda3\Lib\site-packages\sklearn\base.py", line 1474, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\yeotaekyeong\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py", lin

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import ParameterGrid, train_test_split, cross_val_score
from sklearn.metrics import accuracy_score
import pandas as pd
from tqdm.auto import tqdm
from joblib import Parallel, delayed

# 데이터 로드 및 전처리
df = pd.read_csv("하이닉스 power (3).csv", thousands=',', encoding='cp949')
df['next_day_return'] = (df['종가'].shift(-1) - df['종가']) / df['종가'] * 100
df['target'] = df['next_day_return'].apply(lambda x: 1 if x > 0.25 else 0)
df.dropna(inplace=True)

X = df.drop(['날짜', 'target', 'next_day_return'], axis=1)
y = df['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

# 로지스틱 회귀 모델을 위한 하이퍼파라미터 그리드 정의
param_grid_lr = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'penalty': ['l1', 'l2', 'elasticnet', 'none'],
    'solver': ['saga']
}

# 평가 함수
def evaluate_model(params):
    model = LogisticRegression(max_iter=10000, random_state=1, **params)
    scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')
    return scores.mean()

# 진행바와 함께 모든 하이퍼파라미터 조합에 대해 평가
results = Parallel(n_jobs=-1)(
    delayed(evaluate_model)(params) for params in tqdm(list(ParameterGrid(param_grid_lr)), desc='Logistic Regression Tuning')
)

# 최적의 하이퍼파라미터 조합 찾기
best_score_idx = np.argmax(results)
best_params = list(ParameterGrid(param_grid_lr))[best_score_idx]
print(f"Best parameters: {best_params}")
print(f"Best cross-validation score: {results[best_score_idx]:.4f}")

# 최적의 하이퍼파라미터로 모델 학습
best_model = LogisticRegression(max_iter=10000, random_state=1, **best_params)
best_model.fit(X_train, y_train)

# 최적의 모델로 테스트 데이터 예측
y_pred = best_model.predict(X_test)

# 예측 결과에 대한 평가 지표 출력
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
